In [8]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from unicodedata import digit

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("balanced_urls.csv")
df.head()

,url,label,result
0,https://www.google.com,benign,0
1,https://www.youtube.com,benign,0
2,https://www.facebook.com,benign,0
3,https://www.baidu.com,benign,0
4,https://www.wikipedia.org,benign,0


In [ ]:
def lenght_url(url):
  return len(url)

In [ ]:
def length_host(url):
  parsed = urlparse(url)
  
  if not parsed.netloc:
    if "www" == parsed.path[0:3]:
      stops = parsed.path[4:]
      return stops.find("/")
    else:
      stops = parsed.path.find("/")
      return stops
  else:
    if "www" == parsed.netloc[0:3]:
      stops = parsed.netloc[4:]
      return len(stops)
    else:
      return len(parsed.netloc)

In [ ]:
def length_path(url):
    parsed = urlparse(url)
    if not parsed.netloc:
        path = parsed.path.split("/")
        path.pop(0)
        counter = 0
        for i in path:
          counter+=len(i)
        return counter + len(path)
    else:
        return len(parsed.path)

In [ ]:
def length_first_dir(url):
    parsed = urlparse(url)
    if not parsed.netloc:
        fist_dir = parsed.path.split("/")
        if len(fist_dir) != 1:
          return len(fist_dir[1])
        else:
          return 0
    else:
        if parsed.path == "":
          return 0
        else:
          fist_dir = parsed.path[1:].split("/")
          return len(fist_dir[0])

In [ ]:
def length_top_dom(url):
  parsed = urlparse(url)
  if not parsed.netloc:
    stops = parsed.path.split("/")[0]
    return len(stops.split(".")[-1])
  else:
    return len(parsed.netloc.split(".")[-1])

NameError: ignored

In [ ]:
def is_url_shortened(url):
    regex = 'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|' 'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|' 'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|' 'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|' 'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|' 'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|' 'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|' 'tr\.im|link\.zip\.net'
    short = re.search(regex,url)
    if short:
      return 1
    else:
      return 0

In [ ]:
def count_feat(url):
  feat = {"dash":0,"at":0,"questions":0,"percentage":0,"fullstop":0,"equal":0,"http":0,"https":0,"www":0,"digits":0,"letters":0,"directories":0}

  for i in url:
    if i=="-" : feat["dash"] += 1
    elif i=="@" : feat["at"] += 1
    elif i=="?" : feat["questions"] += 1
    elif i=="%" : feat["percentage"] += 1
    elif i=="." : feat["fullstop"] += 1
    elif i=="=" : feat["equal"] += 1 
    elif i.isalpha() : feat['letters'] +=1
    elif i in '1234567890' : feat['digits'] +=1
  k=0
  if "http:/" in url:
    feat["http"] += 1
    k=1
  if "https:/" in url:
    feat["https"] += 1
    k=1
  if "www." in url:
    feat["www"] += 1
  dir = url.split('/')
  if '' in dir:dir.remove('')
  feat['directories']=len(dir)-k
  return feat

In [ ]:
y = df["result"].to_numpy()

def generate_set(df):
  size = len(df)
  x = np.zeros((size,19))
  for i in tqdm(range(size)):
    url = df["url"][i]
    feat = count_feat(url)
    x[i][0],x[i][1],x[i][2],x[i][3] = lenght_url(url),length_host(url),length_path(url),length_first_dir(url)
    x[i][4],x[i][5],x[i][6],x[i][7] = length_top_dom(url),has_ip(url),is_url_shortened(url),feat["directories"]
    x[i][8],x[i][9],x[i][10],x[i][11] = feat["dash"],feat["at"],feat["questions"],feat["percentage"]
    x[i][12],x[i][13],x[i][14],x[i][15] =feat["fullstop"],feat["equal"],feat["http"],feat["https"]
    x[i][16],x[i][17],x[i][18] =feat["www"],feat["digits"],feat["letters"]
  return x

NameError: ignored

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
x = generate_set(df)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.3, random_state=42)

x_train, x_test = x_train_scaled = scaler.fit_transform(x_train),scaler.transform(x_test)

log_model = LogisticRegression(solver='liblinear')
log_model.fit(x_train,y_train)

log_predictions = log_model.predict(x_test)
print(accuracy_score(y_test,log_predictions))

100%|██████████| 632508/632508 [00:37<00:00, 17056.49it/s]


0.9962575323654563


In [ ]:
pata = pd.read_csv("/content/malicious_phish.csv")

In [ ]:
testing = pata.head(10000)
print(testing)

                                                    url        type
0                                      br-icloud.com.br    phishing
1                   mp3raid.com/music/krizz_kaliko.html      benign
2                       bopsecrets.org/rexroth/cr/1.htm      benign
3     http://www.garage-pirenne.be/index.php?option=...  defacement
4     http://adventure-nicaragua.net/index.php?optio...  defacement
...                                                 ...         ...
9995         movies.tvguide.com/eastern-promises/285629      benign
9996  http://themeforest.net/item/moksha-one-multi-p...      benign
9997  flaggedrevs.labs.wikimedia.org/wiki/New_York_S...      benign
9998                thetakeaway.org/people/victor-begg/      benign
9999  thesaurus1.classic.reference.com/roget/VI/898....      benign

[10000 rows x 2 columns]


In [ ]:
replacements = {'benign':0,"phishing":1,"defacement":1,"malware":1}
testing['type'] = testing['type'].replace(replacements)

<ipython-input-76-192102f8cfc6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing['type'] = testing['type'].replace(replacements)


,url,type
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1
...,...,...
9995,movies.tvguide.com/eastern-promises/285629,0
9996,http://themeforest.net/item/moksha-one-multi-p...,0
9997,flaggedrevs.labs.wikimedia.org/wiki/New_York_S...,0
9998,thetakeaway.org/people/victor-begg/,0


In [ ]:
xtesting = generate_set(testing)
ytesting = testing["type"].to_numpy()

100%|██████████| 10000/10000 [00:00<00:00, 20130.70it/s]


In [ ]:
log_predictions = log_model.predict(xtesting)
print(accuracy_score(ytesting,log_predictions))

0.2706
